# `JSONASOBJ2`
An extension to the python `json` library that represents the JSON as a first class python object rather than a straight dictionary. Contents can still be accessed using dictionary format.


## Introduction

This is an extension to the python *json* library that represents the JSON
as a first class python object rather than a straight
dictionary.  Contents can still be accessed using dictionary format.

---
## Requirements

* Python (3.6 or later)

## Installation

```bash
pip install jsonasobj2
```

## Short Example
--------------



In [14]:
!pip install -q  ..

In [15]:
import jsonasobj2
from pprint import PrettyPrinter
pp = PrettyPrinter().pprint

test_json = """{
  "@context": {
    "name": "http://xmlns.com/foaf/0.1/name",
    "knows": "http://xmlns.com/foaf/0.1/knows",
    "menu": {
      "@id": "name:foo",
      "@type": "@id"
    }
  },
  "@id": "http://me.markus-lanthaler.com/",
  "name": "Markus Lanthaler",
  "knows": [
    {
      "name": "Dave Longley",
      "menu": "something",
      "modelDate" : "01/01/2015"
    }
  ]
}"""

py_obj = jsonasobj.loads(test_json)
py_obj.knows[0].extra = {'age': 17}
py_obj.knows.append(dict(name='Barack Obama'))
del py_obj.knows[0]['menu']
print(py_obj.name)
print(py_obj['name'])
print(py_obj.knows[0].name)
print(py_obj['@context'].name)
print(jsonasobj.as_json(py_obj))
pp(jsonasobj.as_dict(py_obj))


Markus Lanthaler
Markus Lanthaler
Dave Longley
http://xmlns.com/foaf/0.1/name
{
   "@context": {
      "name": "http://xmlns.com/foaf/0.1/name",
      "knows": "http://xmlns.com/foaf/0.1/knows",
      "menu": {
         "@id": "name:foo",
         "@type": "@id"
      }
   },
   "@id": "http://me.markus-lanthaler.com/",
   "name": "Markus Lanthaler",
   "knows": [
      {
         "name": "Dave Longley",
         "modelDate": "01/01/2015",
         "extra": {
            "age": 17
         }
      },
      {
         "name": "Barack Obama"
      }
   ]
}
{'@context': {'knows': 'http://xmlns.com/foaf/0.1/knows',
              'menu': {'@id': 'name:foo', '@type': '@id'},
              'name': 'http://xmlns.com/foaf/0.1/name'},
 '@id': 'http://me.markus-lanthaler.com/',
 'knows': [{'extra': {'age': 17},
            'modelDate': '01/01/2015',
            'name': 'Dave Longley'},
           {'name': 'Barack Obama'}],
 'name': 'Markus Lanthaler'}


# Usage notes
JsonObj exposes all properties in the model that don't begin with a `"_"`, which prevents it from behaving like a real dictionary.  This means:

1) The dictionary methods, `list()`, `len()`, `iter()`, `clear()`, `copy()`, `get()`, `items()`, `keys()`, `pop()`, `popitem()`, `reversed()`, `setdefault()`, `update()`, and `values()` along with some of the special operations (`del`, `|`, `|=`) can not be directly exposed.

2) _Because_ these methods can't be exposed, the call `isinstance(obj, dict)` returns `False`.  Use the following instead:

In [18]:
from jsonasobj2 import JsonObj, as_dict, as_json

sample = JsonObj(name="Fred", age=42)
print(isinstance(sample, dict))
print(isinstance(sample, (dict, JsonObj)))

False
True


In [20]:
# Lists exhibit much the same behavior
sample2 = JsonObj([dict(name="John", age=12), "pint of milk"])
print(isinstance(sample2, list))
print(isinstance(sample2, (list, JsonObj)))
print(str(as_dict(sample2)))
print(as_json(sample2))

False
True
[{'name': 'John', 'age': 12}, 'pint of milk']
[
   {
      "name": "John",
      "age": 12
   },
   "pint of milk"
]



## Wrapper methods

We have attempted to mitigate this issue with the following methods:

__as_dict(obj: Union[JsonObj, List]) -> Union[List, Dict[str, JsonTypes]]::__

    Returns obj as a dict or list (Note that a json object can also be a list)
 
 
__as_json(obj: Union[Dict, JsonObj, List], indent: Optional[str] = '   ', filtr: Callable[[Dict], Dict] = None, **kwargs) -> str:__
          
    Returns obj as a JSON String
   
   
__as_json_obj(obj: Union[Dict, JsonObj, List]) -> JsonTypes:__

    Returns obj as a pure _json_ object
    
    
The following methods directly address some, but not all, of the methods above.  If a wrapper isn't available, use `as_dict(obj).method()`

__get(obj: Union[Dict, JsonObj], item: str, default: JsonObjTypes = None) -> JsonObjTypes:__

    Same as obj.get(item, default)
    
    
__setdefault(obj: Union[Dict, JsonObj], k: str, value: Union[Dict, JsonTypes]) -> JsonObjTypes:__

    Same as obj.setdefault(k, value)


__keys(obj: Union[Dict, JsonObj]) -> Iterator[str]:__

    Same as obj.keys()


__items(obj: Union[Dict, JsonObj]) -> Iterator[Tuple[str, JsonObjTypes]]:__

    Same as obj.items()


__values(obj: Union[Dict, JsonObj]) -> Iterator[JsonObjTypes]:__

    Same as obj.values()